# En este notebook se llevarán a cabo los modelos más simples. Solo se trabajará con las variables numéricas del DF y se observará el comportamiento para tener un punto de partida

In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', None)
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [4]:
train = pd.read_csv("/Users/sebascordova/Desktop/Kaggle - House Prices/data/train.csv")
test = pd.read_csv("/Users/sebascordova/Desktop/Kaggle - House Prices/data/test.csv")

In [5]:
print(train.shape)
print(test.shape)

(1460, 81)
(1459, 80)


In [6]:
train["train"] = 1
train["test"] = 0
test["test"] = 1
test["train"] = 0

df = pd.concat([train,test])

variables_n = df.loc[:, df.dtypes == int]

train_1 = variables_n[variables_n["train"]==1]
test_1 = variables_n[variables_n["test"]==1]

train_1 = train_1.join(train[["SalePrice"]])
del(train_1["train"])
del(train_1["test"])
del(test_1["train"])
del(test_1["test"])

print(train_1.shape)
print(test_1.shape)

(1460, 27)
(1459, 26)


## Primer modelo, RF base

In [7]:
model_1 = RandomForestRegressor()
model_1.fit(train_1.loc[:, ~train_1.columns.isin(['SalePrice'])], train["SalePrice"])

RandomForestRegressor()

In [ ]:
y_pred = model_1.predict(test_1)
test_1["SalePrice"] = y_pred
modelo_1 = test_1[["Id","SalePrice"]]
#modelo_1.to_csv("Modelo_base-modelo_1.csv", index=False)

### Con este modelo se consigue un score 0.15587, posición 2596 de 4259 participantes. Nada mal para haber lanzado un modelo simple.

## Se prueba a modificar los hiperparametros del modelo_1 (por defecto n_estimators = 100 y max_depth=None)

In [ ]:
model_2 = RandomForestRegressor(n_estimators = 300, max_depth=7)
model_2.fit(train_1.loc[:, ~train_1.columns.isin(['SalePrice'])], train["SalePrice"])

y_pred = model_2.predict(test_1)
test_1["SalePrice"] = y_pred
model_2 = test_1[["Id","SalePrice"]]
#model_2.to_csv("modelo_2.csv", index=False)

### Con este modelo las predicciones son peores (score de 0.16279)

# Se cambia de modelo, ahora probaremos con XGBoost

In [ ]:
model_3 = XGBRegressor()
model_3.fit(train_1.loc[:, ~train_1.columns.isin(['SalePrice'])], train["SalePrice"])

y_pred = model_3.predict(test_1)
test_1["SalePrice"] = y_pred
model_3 = test_1[["Id","SalePrice"]]
#model_3.to_csv("model_3.csv", index=False)

### Score de 0.17095, de nuevo, el mejor modelo sigue siendo el RF más simple

## Por último, se prueba a tocar los hiperparametros de XGB

In [ ]:
model_4 = XGBRegressor(n_estimators=350, max_depth=7, eta=0.3)
model_4.fit(train_1.loc[:, ~train_1.columns.isin(['SalePrice'])], train["SalePrice"])

y_pred = model_4.predict(test_1)
test_1["SalePrice"] = y_pred
model_4 = test_1[["Id","SalePrice"]]
#model_4.to_csv("model_4.csv", index=False)


###  Score 0.17198, pero modelo hasta el momento

# Se guarda el mejor modelo de los 4, con nombre modelo_1